In [21]:
import pandas as pd
import numpy as np
import warnings
import time
import re

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

In [22]:
# Function 1: Check if Opioid was Taken in the Week After Delivery
def check_opioid_after_delivery_2(df: pd.DataFrame,
                                  delivery_dt: str = "delivery_dt",
                                  opioid_dates: str = "opioid_dates",
                                  opioid_week_ad: str = "opioid_week_ad"):
    """
    Checks if the patient received opioids in the week after delivery.
    
    Returns
    -------
    pd.DataFrame
        A DataFrame with an additional column indicating if the patient received opioids in the week after delivery.
    """
    result_df = pd.DataFrame()

    for index, row in df.iterrows():
        delivery_dt_7 = row[delivery_dt] + pd.Timedelta(days=7)
        unique_dates = row[opioid_dates].split(', ')
        
        opioid_dts = [pd.to_datetime(i.strip(" Timestamp('')")) for i in unique_dates]
        
        opioid_in_p = 'No'
        for date in opioid_dts:
            if row[delivery_dt] <= date <= delivery_dt_7:
                opioid_in_p = 'Yes'
                break
        row[opioid_week_ad] = opioid_in_p
        result_df = pd.concat([result_df, pd.DataFrame(row).transpose()])
    
    return result_df

# Function 2: Extract Opioids
def extract_opioids(name, strength):
    # Ensure name and strength are strings
    if not isinstance(name, str):
        return None, None
    if not isinstance(strength, str):
        strength = str(strength)

    # Define opioid name patterns
    opioid_patterns = ["codeine", "fentanyl", "hydrocodone", "hydromorphone",
                       "morphine", "oxycodone", "tramadol", "cod"]

    # Normalize name to lowercase for matching
    name_lower = name.lower()
    opioid_name = next((op for op in opioid_patterns if op in name_lower), None)

    # Extract all numeric values from the strength string
    try:
        numbers = re.findall(r"\d+\.?\d*", strength)
        numbers = [float(n) for n in numbers]
        opioid_strength = min(numbers) if numbers else None
    except:
        opioid_strength = None

    return opioid_name, opioid_strength


# Function 3: Calculate MMEs with Updated Parameters
def calculate_mme(row,
                  mme_conversion_factor='MME_Conversion_Factor',
                  strength_op='strength_op',
                  op_quan='op_quan',
                  op_dayssup='op_dayssup'):
    """
    Calculates the morphine milligram equivalent (MME) of a prescription.
    
    Returns
    -------
    float
        MME of the prescription.
    """
    if row[op_dayssup] == 0:
        return np.nan
    else:
        mme = row[strength_op] * (row[op_quan] / row[op_dayssup]) * row[mme_conversion_factor]
        return round(mme, 2)

Data load

In [4]:
# 3.1: NDC Code Description
ndc_file_path = r"Z:\lookup\pp_rx_lookup.csv"

# Load NDC codes with product and generic names
ndc_codes_all = pd.read_csv(ndc_file_path, dtype={'ndc': str})
ndc_codes_all = ndc_codes_all[['ndc', 'product_name', 'generic_name','strength']]

# Rename column for consistency
ndc_codes_all = ndc_codes_all.rename(columns={'ndc': 'presc_opioid'})
ndc_codes_all['presc_opioid'] = ndc_codes_all['presc_opioid'].astype(str)
ndc_codes_all

,presc_opioid,product_name,generic_name,strength
0,00093318620,PENCICLOVIR,PENCICLOVIR,1%
1,00121063810,GUAIFENESIN-DM,DEXTROMETHORPHAN-GUAIFENESIN,10-100 MG/5 ML
2,43353024228,ZOLPIDEM TARTRATE,ZOLPIDEM TARTRATE,5 MG
3,43353061260,DOXAZOSIN MESYLATE,DOXAZOSIN MESYLATE,4 MG
4,51407041501,BUSPIRONE HYDROCHLORIDE,BUSPIRONE HCL,10 MG
...,...,...,...,...
189451,40986002163,SMOOTH ANTACID EXTRA STRE,CALCIUM CARBONATE (ANTACID),750 MG
189452,51927225800,AZATHIOPRINE,AZATHIOPRINE,NaN
189453,54868079600,DEPO-TESTOSTERONE,TESTOSTERONE CYPIONATE,100 MG/ML
189454,55953067340,AMOXICILLIN,AMOXICILLIN,250 MG/5 ML


In [5]:
# 3.2: MME Conversion Factor
mme_file_path = r"Z:\chelsea\datalake\final_codes\mme_conversion_factor.csv"

# Load MME conversion factors
conversion_factors = pd.read_csv(mme_file_path)
conversion_factors

,product_name,MME_Conversion_Factor
0,ABSTRAL,0.13
1,ACETAMINOPHEN-CODEINE #4,0.15
2,ACETAMINOPHEN/CODEINE,0.15
3,ACETAMINOPHEN/CODEINE #3,0.15
4,ACETAMINOPHEN/CODEINE #4,0.15
...,...,...
93,XARTEMIS XR,1.50
94,XODOL,1.00
95,XTAMPZA ER,1.50
96,ZAMICET,1.00


In [ ]:
# 3.3: Cohort
cohort_file_path = r"Z:/chelsea/datalake/New_cohort/New_filters/cohort_delivery_type_3.csv"

# Load cohort data and sort by patient ID and from date
data = pd.read_csv(cohort_file_path)
data = data.sort_values(by=['pat_id_p', 'from_dt'])

# Remove duplicates
data = data.drop_duplicates()
print("Cohort Data Sample:\n", data.head())

In [ ]:
# Step 4.1: Group by `pat_id_p` and aggregate values as unique lists, including `pat_state`
data_count = data.groupby('pat_id_p').agg({
    'pat_id': 'unique',
    'der_yob': 'unique', 
    'total_pregnancies': 'unique',
    'type_of_delivery': 'unique',
    'pregnancy_dx_dt': 'unique',
    'delivery_dt': 'unique',
    'days_between_pd': 'unique',
    'last_pat_dt': 'unique',
    'opioid_dates': 'unique', 
    'presc_opioid': 'unique',
    'op_dayssup': 'unique',
    'op_quan': 'unique',
    'pat_state': 'unique',
    'pat_Zip3': 'unique'
})

data_count = data_count.reset_index()
data_count


In [ ]:
data_count.columns

In [ ]:
columns_df = ['pat_id', 'der_yob', 'type_of_delivery', 'total_pregnancies', 'pregnancy_dx_dt',
              'delivery_dt', 'days_between_pd', 'last_pat_dt', 'pat_state','pat_Zip3',
              'opioid_dates', 'presc_opioid', 'op_dayssup', 'op_quan']

# Extract the first element from lists in each specified column
for column in columns_df:
    data_count[column] = data_count[column].apply(lambda x: x[0])

# Further process specific columns to split string representations of lists into actual lists
data_count['opioid_dates'] = data_count['opioid_dates'].apply(lambda x: x.strip("[]").split(','))
data_count['presc_opioid'] = data_count['presc_opioid'].apply(lambda x: x.strip("['']").split(','))
data_count['op_dayssup'] = data_count['op_dayssup'].apply(lambda x: x.strip("[]").split(','))
data_count['op_quan'] = data_count['op_quan'].apply(lambda x: x.strip("[]").split(','))

# Display the resulting DataFrame
data_count


In [ ]:
data_expanded = data_count.explode(['opioid_dates', 'presc_opioid', 'op_dayssup', 'op_quan'])
data_expanded['presc_opioid'] = data_expanded['presc_opioid'].str.lstrip(" '")
data_expanded['presc_opioid'] = data_expanded['presc_opioid'].str.rstrip("' ")
data_expanded['presc_opioid'] = data_expanded['presc_opioid'].astype(float).astype(int).astype(str)
data_expanded = data_expanded.reset_index(drop=True)

# Display the resulting DataFrame
data_expanded


In [ ]:
data_expanded.columns




In [12]:
data_expanded.to_csv(r"Z:\chelsea\datalake\New_cohort\New_filters\cleaned_dataset2.csv", index=False)

In [ ]:
main_data_path = r"Z:\chelsea\datalake\New_cohort\New_filters\cleaned_dataset2.csv"
main_data = pd.read_csv(main_data_path, usecols=['pat_id_p', 'presc_opioid'])

# Load the NDC codes file
ndc_file_path = r"Z:\lookup\pp_rx_lookup.csv"
ndc_codes = pd.read_csv(ndc_file_path, usecols=['ndc', 'product_name', 'generic_name', 'strength'])

# Ensure columns are formatted as strings to avoid mismatches
main_data['presc_opioid'] = main_data['presc_opioid'].astype(str)
ndc_codes['ndc'] = ndc_codes['ndc'].astype(str)

# Rename the ndc column in the NDC file to match the `presc_opioid` column for the merge
ndc_codes = ndc_codes.rename(columns={'ndc': 'presc_opioid'})

# Merge the main data with the NDC file on `presc_opioid`
merged_data = pd.merge(main_data, ndc_codes, how='left', on='presc_opioid')

# Display the merged result
print(merged_data.head())

# Optionally, save the merged result to a new file
merged_data.to_csv("merged_output.csv", index=False)

In [ ]:
main_data_path = r"Z:\chelsea\datalake\New_cohort\New_filters\cleaned_dataset2.csv"
ndc_file_path = r"Z:\lookup\pp_rx_lookup.csv"

# Load the main data file and the NDC codes file
main_data = pd.read_csv(main_data_path)
ndc_codes = pd.read_csv(ndc_file_path, usecols=['ndc', 'product_name', 'generic_name', 'strength'])

# Ensure the columns are formatted as strings to avoid mismatches
main_data['presc_opioid'] = main_data['presc_opioid'].astype(str)
ndc_codes['ndc'] = ndc_codes['ndc'].astype(str)

# Rename the 'ndc' column in the NDC file to match the 'presc_opioid' column for the merge
ndc_codes = ndc_codes.rename(columns={'ndc': 'presc_opioid', 
                                      'product_name': 'ProductName', 
                                      'generic_name': 'GenericName', 
                                      'strength': 'Strength'})

# Merge the main data with the NDC file on 'presc_opioid'
merged_data = pd.merge(main_data, ndc_codes, how='left', on='presc_opioid')

# Display the merged result to check if all columns are retained
print(merged_data.head())
print(merged_data.columns.tolist())

In [ ]:
merged_data

Extracing opioids types and strength

In [ ]:
merged_data['Strength'] = merged_data['Strength'].astype(str)
merged_data[['generic_name_op','strength_op']] = merged_data.apply(
    lambda row: pd.Series(extract_opioids(row['GenericName'],row['Strength'])),
    axis=1
)

merged_data

In [ ]:
merged_data[merged_data['strength_op'].isna()][['GenericName', 'Strength']].value_counts().reset_index(name='count')


In [ ]:
opioid_types = {
    "cod": "codeine",
}

# Apply the replacement in the generic_name_op column
merged_data["generic_name_op"] = merged_data["generic_name_op"].replace(opioid_types, regex=False)

merged_data

In [ ]:
print(merged_data['ProductName'].unique())

In [ ]:
print(conversion_factors['product_name'].unique())

In [28]:
merged_data = merged_data.rename(columns={'ProductName':'product_name'})

In [30]:
merged_data['product_name'] = merged_data['product_name'].astype(str) 



In [31]:
conversion_factors['product_name']=conversion_factors['product_name'].astype(str)

In [ ]:
merged_data = pd.merge(merged_data, conversion_factors, on="product_name")
merged_data

In [ ]:
columns_df = ["op_dayssup", "op_quan", "strength_op", "MME_Conversion_Factor"]

# Clean and convert each specified column
for column in columns_df:
    # Remove any surrounding single quotes or whitespace and then convert to float
    merged_data[column] = merged_data[column].apply(lambda x: str(x).strip(" '"))
    merged_data[column] = merged_data[column].apply(lambda x: float(x) if x else None)

merged_data

Calcaulting MME

In [ ]:
merged_data['mme'] = merged_data.apply(calculate_mme, axis=1)

merged_data

In [36]:
merged_data['delivery_dt']= pd.to_datetime(merged_data['delivery_dt'])

In [37]:
merged_data = check_opioid_after_delivery_2(merged_data)

In [ ]:
merged_data

In [ ]:
merged_data['delivery_dt']= pd.to_datetime(merged_data['delivery_dt'],errors='coerce')
merged_data['delivery_dt'] = merged_data['delivery_dt'].dt.strftime('%Y-%m-%d')
merged_data

In [ ]:
opioid_ndc_file_path = r"Z:\chelsea\datalake\final_codes\ndc_opioids_codes.csv"
opioid_ndc_codes = pd.read_csv(opioid_ndc_file_path)

# Ensure both columns are formatted as strings to avoid mismatches
merged_data['presc_opioid'] = merged_data['presc_opioid'].astype(str)
opioid_ndc_codes['ndc'] = opioid_ndc_codes['ndc'].astype(str)

# Check if each 'presc_opioid' in merged_data matches any NDC code in opioid_ndc_codes
merged_data['opioid_prescribed'] = merged_data['presc_opioid'].apply(
    lambda x: "Yes" if x in opioid_ndc_codes['ndc'].values else "No"
)

# Display the result
merged_data.head()

In [41]:
merged_data.to_csv(r"Z:\chelsea\datalake\New_cohort\New_filters\cohort_opioid2.csv",) 

In [ ]:
file_path = r"Z:\chelsea\datalake\New_cohort\New_filters\cohort_opioid2.csv"
merged_data = pd.read_csv(file_path)

# Recalculate MME based on the formula (strength_op * op_quan / op_dayssup) * MME_Conversion_Factor
merged_data['mme'] = (merged_data['strength_op'] * merged_data['op_quan'] / merged_data['op_dayssup']) * merged_data['MME_Conversion_Factor']

merged_data

In [ ]:
merged_data.columns

In [ ]:
columns_to_keep = [
    'pat_id_p', 'pat_id', 'der_yob', 'total_pregnancies',
       'type_of_delivery', 'pregnancy_dx_dt', 'delivery_dt', 'days_between_pd',
       'last_pat_dt', 'opioid_dates', 'presc_opioid', 'op_dayssup', 'op_quan',
       'pat_state', 'pat_Zip3', 'product_name', 'GenericName', 'Strength',
       'generic_name_op', 'strength_op', 'MME_Conversion_Factor', 'mme',
       'opioid_week_ad', 'opioid_prescribed'
]

# Update merged_data to only include the specified columns
merged_data = merged_data[columns_to_keep]

# Display the updated DataFrame
merged_data


In [48]:
merged_data.to_csv(r"Z:\chelsea\datalake\New_cohort\New_filters\cohort_opioid2.csv", index=False) 

In [ ]:
merged_data.columns